In [4]:
# please put all the package used here
from collections import Counter
import pandas as pd
import csv
import pprint
import numpy as np
import warnings
import requests
import json
warnings.filterwarnings('ignore')
import psycopg2
import psycopg2.extras
import time
import ast # convert string to dictionary
import requests
from bs4 import BeautifulSoup



In [5]:
## scraping data from table of wikipedia

In [99]:
# retrieve the source code of the webpage
page_html = requests.get("https://en.wikipedia.org/wiki/List_of_Sydney_Trains_railway_stations").text
base_url = 'http://maps.googleapis.com/maps/api/geocode/json'
#print(page_html)

In [101]:
page_content = BeautifulSoup(page_html, 'html5lib')
#passengers = page_content.find_all("table",{"class": "wikitable sortable jquery-tablesorter"})
tables = page_content.find(id="bodyContent").find_all("table",{"class": "wikitable sortable"})
#for table in tables:
    #print(table)


In [111]:
def location_boundry(name1):
    try:
        name1 = name1.strip().replace('\n','')
        name1 = name1 +" - Sydney"
        my_params= {'address': name1,'language':'en'}  
        response = requests.get(base_url, params = my_params)
        results  = response.json()['results']
        nhood_bounds = results[0]['geometry']['viewport']
        name1 = name1.replace(" - Sydney","")
        #print(name1)
        #print(results)
        x_geo    = results[0]['geometry']['location']
        if name1 in GPS.keys() or name1 in Location.keys():
            #print("duplication !!! 🐳🐳🐳")
            return
        else:
            GPS[name1] = [x_geo['lng'],x_geo['lat']]
            Location[name1] = [json.dumps(nhood_bounds, indent=4)]
        
        #print("{} GPS : {} {}".format(name1, x_geo['lng'], x_geo['lat']))
        #print("Boundary: ", json.dumps(nhood_bounds, indent=4))
        #print('🌀-----------------------------------------------------------------🌀')
    except IndexError as e:
        #print(name1)
        time.sleep( 1 )  # 过于频繁 抓取失败的话，就休息一秒继续抓
        location_boundry(name1) #recursive
        #print('😂-----------------------------------------------------------------😂')

In [112]:
GPS = {}  # nested list
Location = {}
for name in names:
    name = name[0]
    location_boundry(name) 

KeyboardInterrupt: 

In [107]:
import requests
from bs4 import BeautifulSoup

GPS = {}  # nested list
Location = {}
for name in names:
    name = name[0]
    location_boundry(name) 
    
page_content = BeautifulSoup(page_html, 'html5lib')
tables = page_content.find(id="bodyContent").find_all("table",{"class": "wikitable sortable"})
try:
    names = []
    data_rows = page_content.find_all("tr") # table row  
    count = 0
    for row in data_rows[2:]:
        if count ==151:  # it is empty string
            count += 1
            continue
        name = row.a["href"].replace('/wiki/','').replace(',_Sydney','')
        names.append([name])
        #print(name)
        if name =="Helensburgh_railway_station":  # the last one
            break
        count += 1
except TypeError as e:
    print(row)
    print(count)
# write in new file
with open('TrainStation.csv', 'w') as out_file:
    writer = csv.writer(out_file)
    writer.writerow(["name"])
    writer.writerows(names)
# read file
data = pd.read_csv('TrainStation.csv')


In [109]:
data['GPS_lng']=np.nan
data['GPS_lat']=np.nan

# save GPS
count = 0
for j in data['name']:
    for k,v in GPS.items():
        if j == k:
            data['GPS_lng'][count] = v[0]
            data['GPS_lat'][count] = v[1]
    count += 1

In [110]:
data.to_csv("TrainStation.csv",index=False)

In [113]:
data

,name,GPS_lng,GPS_lat
0,Allawah_railway_station,151.114138,-33.969680
1,Arncliffe_railway_station,151.147292,-33.936241
2,Artarmon_railway_station,151.185124,-33.808831
3,Ashfield_railway_station,151.125847,-33.887595
4,Asquith_railway_station,151.108174,-33.688707
5,Auburn_railway_station,151.032920,-33.849223
6,Banksia_railway_station,151.140413,-33.945307
7,Bankstown_railway_station,151.034144,-33.917853
8,Bardwell_Park_railway_station,151.124895,-33.931572
9,Beecroft_railway_station,151.066388,-33.749686


In [94]:
with open('TrainStation.csv', 'w') as out_file:
    writer = csv.writer(out_file)
    writer.writerow(["name"])
    writer.writerows(names)

In [95]:
data = pd.read_csv('TrainStation.csv')

In [96]:
data

,name
0,Allawah_railway_station
1,Arncliffe_railway_station
2,Artarmon_railway_station
3,Ashfield_railway_station
4,Asquith_railway_station
5,Auburn_railway_station
6,Banksia_railway_station
7,Bankstown_railway_station
8,Bardwell_Park_railway_station
9,Beecroft_railway_station
